# Consolidating the NC data set
* July 22


In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("NorthCarolina Data.csv").drop("Unnamed: 0",axis=1)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Selecting the Elections

In [5]:
df.head()
# for col in df.columns:
#     print(col)

#df.ndim

,county_id,county,election_dt,result_type_lbl,result_type_desc,contest_id,contest_title,contest_party_lbl,contest_vote_for,precinct_code,precinct_name,candidate_id,candidate_name,candidate_party_lbl,group_num,group_name,voting_method_lbl,voting_method_rslt_desc,vote_ct
0,2,ALEXANDER,11/03/2020,STD,<NORMAL>,682,ALEXANDER COUNTY BOARD OF COMMISSIONERS,NaN,2,B1,BETHLEHEM #1 ...,1549,Jeffrey P. Peal,REP,2,ELECTION DAY,V,Election Day,235
1,2,ALEXANDER,11/03/2020,STD,<NORMAL>,682,ALEXANDER COUNTY BOARD OF COMMISSIONERS,NaN,2,B1,BETHLEHEM #1 ...,1549,Jeffrey P. Peal,REP,3,ONE STOP,O,Absentee: One-Stop,1345
2,2,ALEXANDER,11/03/2020,STD,<NORMAL>,682,ALEXANDER COUNTY BOARD OF COMMISSIONERS,NaN,2,B1,BETHLEHEM #1 ...,1549,Jeffrey P. Peal,REP,4,ABSENTEE BY MAIL,M,Absentee: By-Mail,133
3,2,ALEXANDER,11/03/2020,STD,<NORMAL>,682,ALEXANDER COUNTY BOARD OF COMMISSIONERS,NaN,2,B1,BETHLEHEM #1 ...,1549,Jeffrey P. Peal,REP,5,PROVISIONAL,P,Provisional,0
4,2,ALEXANDER,11/03/2020,STD,<NORMAL>,682,ALEXANDER COUNTY BOARD OF COMMISSIONERS,NaN,2,B1,BETHLEHEM #1 ...,1549,Jeffrey P. Peal,REP,6,TRANSFER,T,Election Day: Transfer,0


In [6]:
# Select contest
pres_el_df = df.loc[df['contest_title'] == 'US PRESIDENT']

In [7]:
# Select election day
pres_el_df = pres_el_df.loc[pres_el_df['group_name'] == 'ELECTION DAY']

In [8]:
# Select necessary core variables
pres_el_df = pres_el_df[['county', 'precinct_name', 'vote_ct']].copy() 

### Calculate Turnout at precinct level

In [9]:
vote_counts = pres_el_df.groupby(['county','precinct_name'])['vote_ct'].sum()

In [12]:
# vote_counts.iloc[1:50]
vote_counts.head()

county    precinct_name                                               
ALAMANCE  ALBRIGHT                                                        580
          BOONE 5                                                         597
          BURLINGTON 10                                                   302
          BURLINGTON 4                                                    286
          BURLINGTON 5                                                    297
Name: vote_ct, dtype: int64

In [68]:
len(vote_counts)

2454

#### Email with instructions from NCSBE


Hi Chris,
 
You’re on the right track, however if you’ll use the files within the precinct sort folder for the 2020 general election you can calculate the total ballots cast on Election Day using the following logic. First, you’ll want to filter the contest_title to only the “US PRESIDENT” value, and filter the group_name to “ELECTION DAY”. From there you can sum the vote_ct by precinct to get your result.
 
Information about the precinct sort files is available here: Historical Election Results Data | NCSBE
 
Please note the section about statistical noise added to that file.
 
Let us know if you have any further questions.
 
 
Thanks,
Colin Loftin

## Matching the Polling Station Places with the Turnout
#### Two different data sources 
* dat_ps = 'polling_place_20201103.csv'
* dat_vote = 'NorthCarolina_Data.csv'

#### Prepare dat_vote 
* select contest_title' == 'US PRESIDENT'
* select group_name' == 'ELECTION DAY'
* sum votes by precinct 
* resulting object: data frame with county, precinct_name and vote_count

#### Match 
for precinct_name in dat_ps
* lookup county and precinct_name in dat_vote
* get vote_count 
* write 


In [22]:
dat_vote=pres_el_df.groupby('precinct_name').agg({
    # preparing for turnout
    'vote_ct': 'sum',
    'county': 'first',
})

In [23]:
dat_vote

,vote_ct,county
precinct_name,,
001,188,UNION
002,200,UNION
003,115,UNION
004,206,UNION
005,378,UNION
...,...,...
WOODSDALE,196,PERSON
WOODVILLE,91,BERTIE
YANCEYVILLE,243,CASWELL


In [24]:
dat_vote.index.name = 'precinct_name'
dat_vote.reset_index(inplace=True)
# replace the / for a - to be in sync with the image names
dat_vote['precinct_name'] = dat_vote['precinct_name']

In [40]:
dat_vote

,precinct_name,vote_ct,county,common
0,001 ...,188,UNION,UNION_001 ...
1,002 ...,200,UNION,UNION_002 ...
2,003 ...,115,UNION,UNION_003 ...
3,004 ...,206,UNION,UNION_004 ...
4,005 ...,378,UNION,UNION_005 ...
...,...,...,...,...
2367,WOODSDALE ...,196,PERSON,PERSON_WOODSDALE ...
2368,WOODVILLE ...,91,BERTIE,BERTIE_WOODVILLE ...
2369,YANCEYVILLE ...,243,CASWELL,CASWELL_YANCEYVILLE ...
2370,YEOPIM ...,230,CHOWAN,CHOWAN_YEOPIM ...


In [26]:
dat_ps=pd.read_csv("polling_place_20201103.csv")
dat_ps

,election_dt,county_name,polling_place_id,polling_place_name,precinct_name,house_num,street_name,city,state,zip
0,11/03/20,ALAMANCE,1,ALAMANCE CIVITAN CLUB HOUSE,COBLE,3328,DOCTOR PICKETT RD,BURLINGTON,NC,27215
1,11/03/20,ALAMANCE,10,ELMIRA COMMUNITY CENTER,BURLINGTON 7,810,WICKER ST,BURLINGTON,NC,27217
2,11/03/20,ALAMANCE,12,FIRST BAPTIST CHURCH OF ELON,NORTH BOONE,621,HAGGARD AVE,ELON,NC,27244
3,11/03/20,ALAMANCE,13,LAKEVIEW COMMUNITY CHURCH,FAUCETTE,101,BOONE RD,BURLINGTON,NC,27217
4,11/03/20,ALAMANCE,14,FELLOWSHIP BAPTIST CHURCH,GRAHAM 4,2744,MAPLE AVE,BURLINGTON,NC,27215
...,...,...,...,...,...,...,...,...,...,...
2657,11/03/20,YANCEY,42,WEST YANCEY FIRE DEPARTMENT,CANE RIVER,6557,US HWY 19,BURNSVILLE,NC,28714
2658,11/03/20,YANCEY,5,RAMSEYTOWN FIRE DEPT,RAMSEYTOWN,14530,US HWY 19W,BURNSVILLE,NC,28714
2659,11/03/20,YANCEY,52,CLEAMONT ELEMENTARY SCHOOL (05,GREEN MOUNTAIN,1175,CLEARMONT SCHOOL RD,BURNSVILLE,NC,28714
2660,11/03/20,YANCEY,7,CLEARMONT ELEMENTARY SCHOOL,JACKS CREEK,1175,CLEARMONT SCHOOL RD,BURNSVILLE,NC,28714


In [46]:
coun=0
for i in dat_vote.precinct_name.unique():
    for j in dat_ps.precinct_name.unique():
        if i in j:
            similar.append(j)
            coun+=1
            
        

In [47]:
coun

554

In [48]:
coun=0
non_similar=[]
for i in dat_vote.precinct_name.unique():
    for j in dat_ps.precinct_name.unique():
        if i!=j and j not in non_similar:
            non_similar.append(j)
            print(j)
            coun+=1

COBLE
BURLINGTON 7
NORTH BOONE
FAUCETTE
GRAHAM 4
WEST BURLINGTON
EAST GRAHAM
NORTH GRAHAM
BURLINGTON 5
EAST BURLINGTON
BURLINGTON 4
BURLINGTON 8
NORTH BURLINGTON
SOUTH MELVILLE
PATTERSON
GRAHAM 3
SOUTH BURLINGTON
WEST GRAHAM
PLEASANT GROVE
SOUTH THOMPSON
SOUTH BOONE
BURLINGTON 10
NORTH THOMPSON
BURLINGTON 6
NORTH BOONE 2
MELVILLE 3
WEST BOONE
MELVILLE 4
CENTRAL BOONE
ALBRIGHT
BOONE 5
HAW RIVER
MORTON
NORTH MELVILLE
SOUTH GRAHAM
BURLINGTON 9
SOUTH NEWLIN
NORTH NEWLIN
LITTLE RIVER AND SUGAR LOAF
MILLERS
TAYLORSVILLE #1 #4 #5
BETHLEHEM #2
GWALTNEY #1 #2
TAYLORSVILLE #2 #3
ELLENDALE
WITTENBURG
BETHLEHEM #1
SHARPES #1 #2
CHERRY LANE
GLADE CREEK
GAP CIVIL
PRATHERS CREEK
ANSONVILLE
WHITE STORE
BURNSVILLE
GULLEDGE
LILESVILLE
MORVEN/MCFARLAN
PEACHLAND
POLKTON
WADESBORO
JEFFERSON
LAUREL
NEW RIVER
NORTH FORK
0BIDS
OLD FIELDS
PEAK CREEK
PINE SWAMP
POND MOUNTAIN
PINEY CREEK
WEST JEFFERSON
CLIFTON
CRESTON
ELK
HELTON
HURRICANE
HORSE CREEK
ALTAMONT
INGALLS
LINVILLE
MINNEAPOLIS
MONTEZUMA
NEWLAND # 1
NE

In [49]:
coun

2495

In [50]:
dat_vote['common']=dat_vote['county']+"_"+dat_vote['precinct_name']

In [51]:
dat_ps['common']=dat_ps['county_name']+"_"+dat_ps['precinct_name']

In [54]:
coun=0
non_similar=[]
for i in dat_vote.common.unique():
    for j in dat_ps.common.unique():
        if i not in j and j not in non_similar:
            non_similar.append(j)
            print(j)
            coun+=1

ALAMANCE_COBLE
ALAMANCE_BURLINGTON 7
ALAMANCE_NORTH BOONE
ALAMANCE_FAUCETTE
ALAMANCE_GRAHAM 4
ALAMANCE_WEST BURLINGTON
ALAMANCE_EAST GRAHAM
ALAMANCE_NORTH GRAHAM
ALAMANCE_BURLINGTON 5
ALAMANCE_EAST BURLINGTON
ALAMANCE_BURLINGTON 4
ALAMANCE_BURLINGTON 8
ALAMANCE_NORTH BURLINGTON
ALAMANCE_SOUTH MELVILLE
ALAMANCE_PATTERSON
ALAMANCE_GRAHAM 3
ALAMANCE_SOUTH BURLINGTON
ALAMANCE_WEST GRAHAM
ALAMANCE_PLEASANT GROVE
ALAMANCE_SOUTH THOMPSON
ALAMANCE_SOUTH BOONE
ALAMANCE_BURLINGTON 10
ALAMANCE_NORTH THOMPSON
ALAMANCE_BURLINGTON 6
ALAMANCE_NORTH BOONE 2
ALAMANCE_MELVILLE 3
ALAMANCE_WEST BOONE
ALAMANCE_MELVILLE 4
ALAMANCE_CENTRAL BOONE
ALAMANCE_ALBRIGHT
ALAMANCE_BOONE 5
ALAMANCE_HAW RIVER
ALAMANCE_MORTON
ALAMANCE_NORTH MELVILLE
ALAMANCE_SOUTH GRAHAM
ALAMANCE_BURLINGTON 9
ALAMANCE_SOUTH NEWLIN
ALAMANCE_NORTH NEWLIN
ALEXANDER_LITTLE RIVER AND SUGAR LOAF
ALEXANDER_MILLERS
ALEXANDER_TAYLORSVILLE #1 #4 #5
ALEXANDER_BETHLEHEM #2
ALEXANDER_GWALTNEY #1 #2
ALEXANDER_TAYLORSVILLE #2 #3
ALEXANDER_ELLENDALE
AL

In [55]:
coun

2662

In [56]:
coun=0
for i in dat_vote.common.unique():
    for j in dat_ps.common.unique():
        if i in j:
            similar.append(j)
            coun+=1

In [57]:
coun

187